## gensim으로 네이버 기사 토픽 모델링 해보기

> 토픽 모델링을 적용하기 위해 텍스트를 처리합니다.

> 토픽 모델링 라이브러리인 gensim을 사용해봅니다.

### 1. 토픽 모델링을 위한 라이브러리 불러오기

In [1]:
from tqdm import tqdm_notebook # progress bar
from konlpy.tag import Mecab #Mecab, Okt 등 형태소 분석기 불러오기
import numpy as np
import string # 특수문자
import re
import warnings # 경고 알림 제거
import pickle
from gensim import corpora # corpus LDA로 돌릴 수 있는 형태로 변환해주는 기능 
from gensim import models
import matplotlib.pyplot as plt
%matplotlib inline

warnings.filterwarnings("ignore", category=DeprecationWarning) # 경고 알림이 뜨면 모두 무시합니다.

### 2. 텍스트 전처리 함수 만들기

In [2]:
def read_documents(input_file_name):
    """문서들을 주어진 이름의 파일로부터 읽어들여 돌려준다."""
    
    corpus = []
    
    with open(input_file_name, 'rb') as f:
        temp_corpus = pickle.load(f)
        
    for page in temp_corpus:
        corpus += page
    
    return corpus

def text_cleaning(docs):
    # 한국어를 제외한 글자를 제거하는 함수.
    for doc in docs:
        doc = re.sub("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", "", doc)

    return docs

def define_stopwords(path):
    
    SW = set()
    # 불용어를 추가하는 방법 1.
    for i in string.punctuation:
        SW.add(i)

    # 불용어를 추가하는 방법 2.
    # stopwords-ko.txt에 직접 추가
    
    with open(path) as f:
        for word in f:
            SW.add(word)

    return SW


def text_tokenizing(corpus, tokenizer):
    
    mecab = Mecab()
    token_corpus = []
    

    if tokenizer == "noun":
        for n in tqdm_notebook(range(len(corpus)), desc="Preprocessing"):
            token_text = mecab.nouns(corpus[n])
            token_text = [word for word in token_text if word not in SW and len(word) > 1]
                
            token_corpus.append(token_text)
            
    elif tokenized == "morph":
        for n in tqdm_notebook(range(len(corpus)), desc="Preprocessing"):
            token_text = mecab.morphs(corpus[n])
            token_text = [word for word in token_text if word not in SW and len(word) > 1]
            token_corpus.append(token_text)

    elif tokenizer == "word":
        for n in tqdm_notebook(range(len(corpus)), desc="Preprocessing"):
            token_text = corpus[n].split()
            token_text = [word for word in token_text if word not in SW and len(word) > 1]
            token_corpus.append(token_text)
        

    return token_corpus

문서 읽기의 과정은 앞서 단어 임베딩의 경우와 다르지 않다. 다음 과정은 문서-단어 행렬을 만드는 과정이다.

In [3]:
input_file_name = "test_content.pk"
documents = read_documents(input_file_name)
SW = define_stopwords("stopwords-ko.txt")
cleaned_text = text_cleaning(documents)
tokenized_text = text_tokenizing(cleaned_text, tokenizer="noun") #tokenizer= "noun" or "word"

In [4]:
tokenized_text

[['최근',
  '인터넷',
  '개인',
  '방송',
  '매개',
  '선정',
  '폭력',
  '영상',
  '불법',
  '정보',
  '유통',
  '인터넷',
  '개인',
  '방송',
  '규제',
  '강화',
  '지적',
  '제기',
  '인터넷',
  '개인',
  '방송',
  '특성',
  '불법',
  '정보',
  '유통',
  '여부',
  '방송',
  '송신',
  '이후',
  '판단',
  '현행법',
  '인터넷',
  '개인',
  '방송',
  '사업자',
  '송신',
  '영상',
  '콘텐츠',
  '저장',
  '의무',
  '규정',
  '규제',
  '실정',
  '인터넷',
  '개인',
  '방송',
  '사업자',
  '운영',
  '관리',
  '정보',
  '통신망',
  '유통',
  '정보',
  '일정',
  '기간',
  '동안',
  '보관',
  '의무',
  '위반',
  '경우',
  '과태료',
  '부과',
  '인터넷',
  '개인',
  '방송',
  '매개',
  '불법',
  '정보',
  '유통',
  '신설',
  '현행법',
  '복수',
  '진행자',
  '콘텐츠',
  '실시간',
  '정보',
  '통신망',
  '이용자',
  '제공',
  '인터넷',
  '개인',
  '방송',
  '정의',
  '규제',
  '마련',
  '인터넷',
  '개인',
  '방송',
  '도박',
  '성매매',
  '음란',
  '명예훼손',
  '욕설',
  '차별',
  '비하',
  '발언',
  '선정',
  '폭력',
  '방송',
  '발생',
  '불법',
  '정보',
  '청소년',
  '유해',
  '정보',
  '정보',
  '통신망',
  '유통',
  '규제',
  '규제',
  '필요',
  '지속',
  '지적',
  '인터넷',
  '개인',
  '방송',
  '사업자',
  '청소년',
  '보호',
  '책임자',
  '지정',
  '

In [5]:
print(tokenized_text[0])

['최근', '인터넷', '개인', '방송', '매개', '선정', '폭력', '영상', '불법', '정보', '유통', '인터넷', '개인', '방송', '규제', '강화', '지적', '제기', '인터넷', '개인', '방송', '특성', '불법', '정보', '유통', '여부', '방송', '송신', '이후', '판단', '현행법', '인터넷', '개인', '방송', '사업자', '송신', '영상', '콘텐츠', '저장', '의무', '규정', '규제', '실정', '인터넷', '개인', '방송', '사업자', '운영', '관리', '정보', '통신망', '유통', '정보', '일정', '기간', '동안', '보관', '의무', '위반', '경우', '과태료', '부과', '인터넷', '개인', '방송', '매개', '불법', '정보', '유통', '신설', '현행법', '복수', '진행자', '콘텐츠', '실시간', '정보', '통신망', '이용자', '제공', '인터넷', '개인', '방송', '정의', '규제', '마련', '인터넷', '개인', '방송', '도박', '성매매', '음란', '명예훼손', '욕설', '차별', '비하', '발언', '선정', '폭력', '방송', '발생', '불법', '정보', '청소년', '유해', '정보', '정보', '통신망', '유통', '규제', '규제', '필요', '지속', '지적', '인터넷', '개인', '방송', '사업자', '청소년', '보호', '책임자', '지정', '불법', '정보', '삭제', '유통', '차단', '위반', '경우', '과태료', '부과', '정보', '통신망', '불법', '정보', '청소년', '유해', '정보', '유통', '정보', '통신망', '청소년', '유해', '정보', '청소년', '보호', '청소년', '보호', '책임자', '지정', '대상자', '인터넷', '개인', '방송', '사업자', '추가', '인터넷', '개인', '방송', '사업자', '운영

### 3. 토픽 모델링에 사용할 함수들 확인하기

In [6]:
# 문서-단어 행렬 만들기
# 어휘(vocabulary) 학습
dictionary = corpora.Dictionary(tokenized_text)
# 문서-단어 행렬(document-term matrix) 생성
corpus = [dictionary.doc2bow(text) for text in tokenized_text]

In [7]:
print(dictionary)

Dictionary(282 unique tokens: ['가이드라인', '강연료', '강화', '개선', '개인']...)


In [8]:
corpus[0][:5]

[(0, 7), (1, 1), (2, 8), (3, 1), (4, 44)]

In [9]:
# TFIDF 문서-단어 행렬 생성
tfidf = models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]
corpus_tfidf[0][:5]

[]

In [10]:
model = models.ldamodel.LdaModel(corpus_tfidf, num_topics=4, id2word=dictionary)

/Users/yongha/opt/anaconda3/envs/yh/lib/python3.8/site-packages/gensim/models/ldamodel.py:821: RuntimeWarning: divide by zero encountered in double_scalars
  perwordbound = self.bound(chunk, subsample_ratio=subsample_ratio) / (subsample_ratio * corpus_words)


In [11]:
model.show_topic(3, 10)

[('장관', 0.0035460987),
 ('자유', 0.0035460987),
 ('정보', 0.0035460987),
 ('정도', 0.0035460987),
 ('전방', 0.0035460987),
 ('저장', 0.0035460987),
 ('장치', 0.0035460987),
 ('정의', 0.0035460987),
 ('자율', 0.0035460987),
 ('일환', 0.0035460987)]

### 4. 토픽 모델링을 추가하여 코드 완성하기

In [12]:
# 토픽 개수, 키워드 개수를 정해주는 변수를 추가.
NUM_TOPICS = 2

NUM_TOPIC_WORDS = 50


def build_doc_term_mat(documents):
    # 문서-단어 행렬 만들어주는 함수.
    print("Building document-term matrix.")
    dictionary = corpora.Dictionary(documents)
    corpus = [dictionary.doc2bow(document) for document in documents]
        
    return corpus, dictionary


def print_topic_words(model):

    # 토픽 모델링 결과를 출력해 주는 함수.
    print("\nPrinting topic words.\n")
    
    for topic_id in range(model.num_topics):
        topic_word_probs = model.show_topic(topic_id, NUM_TOPIC_WORDS)
        print("Topic ID: {}".format(topic_id))
        
        for topic_word, prob in topic_word_probs:
            print("\t{}\t{}".format(topic_word, prob))
            
        print("\n")

# document-term matrix를 만들고,
corpus, dictionary = build_doc_term_mat(tokenized_text)
# LDA를 실행.
model = models.ldamodel.LdaModel(corpus, num_topics=NUM_TOPICS, id2word=dictionary, alpha="auto", eta="auto")
# 결과를 출력.
print_topic_words(model)

Building document-term matrix.

Printing topic words.

Topic ID: 0
	방송	0.04607158154249191
	정보	0.04459259659051895
	인터넷	0.043995097279548645
	개인	0.03461017832159996
	불법	0.026547877117991447
	유통	0.02382894791662693
	신설	0.016786225140094757
	사업자	0.014886061660945415
	통신	0.013442779891192913
	규제	0.013441822491586208
	통신망	0.01168040931224823
	제공	0.011544237844645977
	청소년	0.009813825599849224
	유해	0.009460910223424435
	신문	0.009273738600313663
	자율	0.009064081124961376
	단체	0.009052373468875885
	최근	0.0082304198294878
	삭제	0.007929583080112934
	이용	0.007591800298541784
	강화	0.0073242271319031715
	정치	0.007004073821008205
	서비스	0.006964326836168766
	활동	0.0069276499561965466
	차단	0.006750568747520447
	조제	0.006699450314044952
	음란	0.006417847704142332
	영상	0.006283437833189964
	시행	0.00606463011354208
	위원회	0.005852587521076202
	가이드라인	0.0058350227773189545
	콘텐츠	0.005802431143820286
	경우	0.00572545500472188
	보호	0.005567374173551798
	발생	0.005553102120757103
	산업	0.0053800917230546474
	지적	0.0051941643469035625
	선

### 5. pyLDAvis를 통한 토픽 모델링 결과 시각화하기

In [13]:
# !pip install pyLDAvis

In [14]:
# pyLDAvis 불러오기
import pyLDAvis
import pyLDAvis.gensim

# pyLDAvis를 jupyter notebook에서 실행할 수 있게 활성화.
pyLDAvis.enable_notebook()

# pyLDAvis 실행.
data = pyLDAvis.gensim.prepare(model, corpus, dictionary)
data

PreparedData(topic_coordinates=              x    y  topics  cluster       Freq
topic                                           
1      0.001969  0.0       1        1  63.192556
0     -0.001969  0.0       2        1  36.807444, topic_info=    Term       Freq      Total Category  logprob  loglift
95    불법  22.000000  22.000000  Default  30.0000  30.0000
176  인터넷  43.000000  43.000000  Default  29.0000  29.0000
137   신설  13.000000  13.000000  Default  28.0000  28.0000
82    방송  47.000000  47.000000  Default  27.0000  27.0000
4     개인  35.000000  35.000000  Default  26.0000  26.0000
..   ...        ...        ...      ...      ...      ...
223   차단   2.517013   7.084267   Topic2  -4.9981  -0.0353
136   신문   3.457800  12.245756   Topic2  -4.6806  -0.2651
205   조제   2.497953   7.095615   Topic2  -5.0057  -0.0445
166   음란   2.392955   7.158128   Topic2  -5.0487  -0.0962
113  서비스   2.596715   8.943984   Topic2  -4.9670  -0.2373

[139 rows x 6 columns], token_table=      Topic      Freq   Term
term                        
0         1  0.631514  가이드라인
0         2  0.315757  가이드라인
2         1  0.574962     강화
2         2  0.431221     강화
4         1  0.624451     개인
...     ...       ...    ...
274       2  0.417072     형성
277       1  0.662977     확산
277       2  0.662977     확산
279       1  0.656652     활동
279       2  0.492489     활동

[168 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 1])